In [1]:
import pandas as pd
import numpy as np
import mglearn



In [2]:
housing_df=pd.read_csv("datasets/housing.csv")
housing_df=housing_df.dropna()

In [3]:
housing_df.head()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [4]:
housing_df['ocean_proximity'].value_counts()

ocean_proximity
<1H OCEAN     9034
INLAND        6496
NEAR OCEAN    2628
NEAR BAY      2270
ISLAND           5
Name: count, dtype: int64

In [5]:
targets=housing_df['median_house_value']
targets.shape

(20433,)

In [6]:
data=housing_df.drop(columns='median_house_value')
data.shape

(20433, 9)

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data,targets,random_state=0)


In [8]:
scalers=data.columns[0:len(data.columns)-1]
scalers

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income'],
      dtype='object')

In [9]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer

clf=ColumnTransformer(
  [("scaling",RobustScaler(),scalers),
   ( "onehot",OneHotEncoder(sparse_output=False),['ocean_proximity'])]
)

In [10]:
clf.fit(X_train)

ColumnTransformer(transformers=[('scaling', RobustScaler(),
                                 Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income'],
      dtype='object')),
                                ('onehot', OneHotEncoder(sparse_output=False),
                                 ['ocean_proximity'])])

In [11]:
X_train_trans=clf.transform(X_train)
X_test_trans=clf.transform(X_test)


In [12]:
from sklearn.linear_model import Ridge
ridge=Ridge(alpha=10)

ridge.fit(X_train_trans,y_train)

Ridge(alpha=10)

In [13]:
print("Training Accuracy: {:.3f}".format(ridge.score(X_train_trans,y_train)))
print("Testing Accuracy: {:.3f}".format(ridge.score(X_test_trans,y_test)))

Training Accuracy: 0.645
Testing Accuracy: 0.648


In [14]:
clf.get_feature_names_out()

array(['scaling__longitude', 'scaling__latitude',
       'scaling__housing_median_age', 'scaling__total_rooms',
       'scaling__total_bedrooms', 'scaling__population',
       'scaling__households', 'scaling__median_income',
       'onehot__ocean_proximity_<1H OCEAN',
       'onehot__ocean_proximity_INLAND', 'onehot__ocean_proximity_ISLAND',
       'onehot__ocean_proximity_NEAR BAY',
       'onehot__ocean_proximity_NEAR OCEAN'], dtype=object)

In [15]:
data_dummies=pd.get_dummies(data).astype(int)

In [16]:
data_dummies.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122,37,41,880,129,322,126,8,0,0,0,1,0
1,-122,37,21,7099,1106,2401,1138,8,0,0,0,1,0
2,-122,37,52,1467,190,496,177,7,0,0,0,1,0
3,-122,37,52,1274,235,558,219,5,0,0,0,1,0
4,-122,37,52,1627,280,565,259,3,0,0,0,1,0


In [17]:
y_train_log=np.log(y_train+1)
y_test_log=np.log(y_test+1)

In [18]:
ridge=Ridge().fit(X_train_trans,y_train_log)
print("Testing Accuracy: {:.2f}".format(ridge.score(X_test_trans,y_test_log)))

Testing Accuracy: 0.67


C:\Users\thami\AppData\Local\Temp\ipykernel_10388\3506069592.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(data.longitude)


longitude
-118.31    159
-118.30    157
-118.29    146
-118.32    141
-118.27    141
          ... 
-123.25      1
-115.88      1
-116.27      1
-116.73      1
-116.28      1
Name: count, Length: 844, dtype: int64